In [3]:
from data_utils import load_dataset, dataset
from pathlib import Path

# All subdir in logs directory
logs_dir = Path('logs')
logs_subdir = [x.name for x in logs_dir.iterdir() if x.is_dir()]

result = [ (subdir, len(load_dataset(subdir)['template'].unique())) for subdir in logs_subdir ]

# sort by number of templates
result.sort(key=lambda x: x[1], reverse=False)
result

[('Apache', 6),
 ('Proxifier', 8),
 ('HDFS', 14),
 ('OpenSSH', 26),
 ('Spark', 36),
 ('OpenStack', 43),
 ('HPC', 46),
 ('Zookeeper', 50),
 ('Windows', 50),
 ('HealthApp', 75),
 ('Hadoop', 114),
 ('Linux', 118),
 ('BGL', 120),
 ('Thunderbird', 149),
 ('Android', 158),
 ('Mac', 341)]

In [7]:
import json
from LogParsingGPT import replace_variable
with open('result_Proxifier.json', 'r') as f:
    result = json.load(f)
for k in result.keys():
    print(f'{len(result[k]["matches"])} : {k}')
for k in result.keys():
    print(replace_variable(k))

354 : {host}:{port} close, {sent_bytes} bytes sent, {received_bytes} bytes received, lifetime {lifetime}
955 : {host}:{port} open through proxy {proxy}:{proxy_port} {protocol}
120 : {host}:{port} close, {sent_bytes} bytes ({int(sent_bytes)/1024:.2f} KB) sent, {received_bytes} bytes received, lifetime {lifetime}
167 : {host}:{port} close, {sent_bytes} bytes sent, {received_bytes} bytes ({int(received_bytes)/1024:.2f} KB) received, lifetime {lifetime}
293 : {host}:{port} close, {sent_bytes} bytes ({int(sent_bytes)/1024:.2f} KB) sent, {received_bytes} bytes ({int(received_bytes)/1024:.2f} KB) received, lifetime {lifetime}
8 : {host}:{port} close, {sent_bytes} bytes ({int(sent_bytes)/1024:.2f} KB) sent, {received_bytes} bytes ({int(received_bytes)/1024/1024:.2f} MB) received, lifetime {lifetime}
96 : {host}:{port} error : {error_msg}
<*>:<*> close, <*> bytes sent, <*> bytes received, lifetime <*>
<*>:<*> open through proxy <*>:<*> <*>
<*>:<*> close, <*> bytes (<*> KB) sent, <*> bytes recei

In [5]:
import json
with open('result_Apache.json', 'r') as f:
    result = json.load(f)
for k in result.keys():
    print(f'{len(result[k]["matches"])} : {k}')

538 : mod_jk child workerEnv in error state {worker_state}
835 : jk2_init() Found child {child_pid} in scoreboard slot {slot_num}
568 : workerEnv.init() ok {path}
31 : [client {client_ip}] Directory index forbidden by rule: {directory_path}
11 : mod_jk child init {init_state} {error_code}
11 : jk2_init() Can't find child {child_pid} in scoreboard
